In [8]:
import os
import torch
from torch.utils.data import Dataset
from transformers import (
    GPT2Config, GPT2LMHeadModel,
    Trainer, TrainingArguments,
    DataCollatorForLanguageModeling, GPT2TokenizerFast
)
from tqdm import tqdm

# === Step 1: Setup Paths ===
chunk_dir = "token_chunks"
model_output_dir = "gpt2_custom_model"
block_size = 512
chunks_to_train = 20  # Only train on first 20 chunks

# === Step 2: Load Tokenizer ===
tokenizer = GPT2TokenizerFast(
    vocab_file="tokenizer_gpt2_custom/vocab.json",
    merges_file="tokenizer_gpt2_custom/merges.txt",
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>"
)

# === Step 3: Define Dataset ===
class ChunkDataset(Dataset):
    def __init__(self, token_ids, block_size=512):
        self.examples = [token_ids[i:i+block_size] for i in range(0, len(token_ids) - block_size, block_size)]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        x = torch.tensor(self.examples[idx])
        return {"input_ids": x, "labels": x.clone()}

# === Step 4: Define GPT2 Model from Scratch ===
config = GPT2Config(
    vocab_size=30000,
    n_positions=512,
    n_ctx=512,
    n_embd=256,
    n_layer=4,
    n_head=4,
)
model = GPT2LMHeadModel(config)

# === Step 5: Training Arguments ===
training_args = TrainingArguments(
    output_dir=model_output_dir,
    overwrite_output_dir=True,
    per_device_train_batch_size=1,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    prediction_loss_only=True,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(model=model, args=training_args, data_collator=data_collator)

# === Step 6: Load and Train on First 20 Chunks ===
chunk_files = sorted([f for f in os.listdir(chunk_dir) if f.endswith(".pt")])[:chunks_to_train]
print(f"🧠 Training on {len(chunk_files)} chunks...")

all_tokens = []

for i, file in enumerate(tqdm(chunk_files, desc="📦 Loading Chunks")):
    file_path = os.path.join(chunk_dir, file)
    tokens = torch.load(file_path)
    all_tokens.extend(tokens)

dataset = ChunkDataset(all_tokens, block_size)
trainer.train_dataset = dataset

print(f"\n🚀 Starting training with {len(dataset)} samples...")
trainer.train()

# === Step 7: Save the Model ===
trainer.save_model(model_output_dir)
print(f"✅ Training complete! Model saved to: {model_output_dir}")


🧠 Training on 20 chunks...


📦 Loading Chunks: 100%|██████████| 20/20 [08:03<00:00, 24.20s/it]



🚀 Starting training with 78124 samples...


Step,Training Loss
100,9.442200
200,8.477900
300,7.913100
400,7.607000
500,7.576100
600,7.577500
700,7.439400
800,7.426700
900,7.469000
1000,7.347300


KeyboardInterrupt: 

In [2]:
import os
import torch
from torch.utils.data import Dataset
from transformers import (
    GPT2Config, GPT2LMHeadModel,
    Trainer, TrainingArguments,
    DataCollatorForLanguageModeling,
    GPT2TokenizerFast
)
from tqdm import tqdm

# === Step 1: Paths and Settings ===
chunk_dir = "token_chunks"
model_output_dir = "gpt2_custom_model"
checkpoint_path = os.path.join(model_output_dir, "checkpoint-56000")
tokenizer_path = "tokenizer_gpt2_custom"
block_size = 512
chunks_to_use = 20

# === Step 2: Load Tokenizer ===
tokenizer = GPT2TokenizerFast.from_pretrained(tokenizer_path)

# === Step 3: Dataset Loader ===
class ChunkDataset(Dataset):
    def __init__(self, token_ids, block_size=512):
        self.examples = [token_ids[i:i+block_size] for i in range(0, len(token_ids) - block_size, block_size)]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        x = torch.tensor(self.examples[idx])
        return {"input_ids": x, "labels": x.clone()}

# === Step 4: Load Model Config & Model ===
config_path = os.path.join(model_output_dir, "config.json")
config = GPT2Config.from_json_file(config_path)
model = GPT2LMHeadModel(config)

# === Step 5: Training Arguments ===
training_args = TrainingArguments(
    output_dir=model_output_dir,
    overwrite_output_dir=False,
    per_device_train_batch_size=1,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    prediction_loss_only=True,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(model=model, args=training_args, data_collator=data_collator)

# === Step 6: Train on 20 Chunks from Checkpoint ===
chunk_files = sorted([f for f in os.listdir(chunk_dir) if f.endswith(".pt")])[:chunks_to_use]
print(f"🧠 Resuming training on {len(chunk_files)} chunks from checkpoint-56000...")

resume_done = False

for i, file in enumerate(chunk_files):
    file_path = os.path.join(chunk_dir, file)
    print(f"\n📦 Loading chunk {i+1}/{len(chunk_files)}: {file}")
    token_ids = torch.load(file_path)
    dataset = ChunkDataset(token_ids.tolist(), block_size)
    trainer.train_dataset = dataset

    if not resume_done and os.path.exists(checkpoint_path):
        print(f"🔁 Resuming from checkpoint: {checkpoint_path}")
        trainer.train(resume_from_checkpoint=checkpoint_path)
        resume_done = True
    else:
        print(f"🚀 Training on chunk {i+1} with {len(dataset)} samples")
        trainer.train()

    print(f"✅ Finished training chunk {i+1}")

# === Step 7: Save Final Model ===
trainer.save_model(model_output_dir)
print(f"\n✅ Training complete! Model saved to: {model_output_dir}")



There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


🧠 Resuming training on 20 chunks from checkpoint-56000...

📦 Loading chunk 1/20: chunk_0.pt
🔁 Resuming from checkpoint: gpt2_custom_model\checkpoint-56000


Step,Training Loss


✅ Finished training chunk 1

📦 Loading chunk 2/20: chunk_1.pt
🚀 Training on chunk 2 with 3906 samples


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


...

✅ Finished training chunk 18

📦 Loading chunk 19/20: chunk_1013.pt
🚀 Training on chunk 19 with 3906 samples


Step,Training Loss
100,5.572200
200,5.592300
300,5.556200
400,5.539400
500,5.551700
600,5.504600
700,5.530800
800,5.531200
900,5.525000
1000,5.508100


✅ Finished training chunk 19

📦 Loading chunk 20/20: chunk_1014.pt
🚀 Training on chunk 20 with 3906 samples


Step,Training Loss
100,5.579900
200,5.554100
300,5.512700
400,5.540500
500,5.567200
600,5.528700
700,5.513700
800,5.498000
900,5.577700
1000,5.555700


✅ Finished training chunk 20

✅ Training complete! Model saved to: gpt2_custom_model
